In [1]:
import pandas as pd
import numpy as np
import os

read the raw data

In [3]:
df=pd.read_csv('total_tran_final.csv')

In [4]:
df.head()

,Unnamed: 0,trd_date,fund_id,investor_id,education,intermediaryid,riskscore,SCRate,TrailerRate,FundRiskScore,...,return-N6M,return-HYG,return-IVV,AUM,Open,Gender,age,income,counts,sub
0,0,2019/8/26,HYG,55887,H.Sch/Dip.,6,5,0.03,0.0,M,...,0.000000,0.000000,0.000000,396.172878,N,M,31,89953.079278,4.554758,N
1,1,2019/10/9,HYG,55887,H.Sch/Dip.,6,5,0.03,0.0,M,...,0.000000,0.003022,0.009479,3820.921478,Y,M,33,103063.060048,44.274871,Y
2,2,2019/11/19,HYG,55887,H.Sch/Dip.,6,5,0.03,0.0,M,...,-0.003604,-0.003570,-0.000223,2003.354360,Y,M,42,71597.614595,23.154813,Y
3,3,2019/12/18,HYG,55887,H.Sch/Dip.,6,5,0.03,0.0,M,...,0.006375,0.001817,0.000063,1657.233641,Y,F,47,53341.786595,18.787367,Y
4,4,2019/12/27,HYG,55887,H.Sch/Dip.,6,5,0.03,0.0,M,...,0.000000,-0.000455,-0.000185,4161.822987,Y,M,46,75276.185086,47.336477,Y


construct a separate dataframe for the total record generation

In [5]:
df1=df.copy()

construct the dummy variable for education 

In [7]:
df1=df1.join(pd.get_dummies(df['education'],prefix='ed'))

In [8]:
del df1['education']

convert fund risk score to ordinal scale

In [9]:
pd.unique(df1.FundRiskScore)

array(['M', 'L', 'H'], dtype=object)

In [10]:
def convfrscore(x):
    if x=='L':
        return 1
    if x=='M':
        return 2
    if x=='H':
        return 3

In [12]:
df1['FRC']=df1.FundRiskScore.apply(convfrscore)

convert gender to numerical type

In [13]:
def gencovert(x):
    if x=='M':
        return 1
    if x=="F":
        return 0

In [14]:
df1.Gender=df1.Gender.apply(gencovert)

convert Open and Sub to numerical type

In [15]:
def yesnoconvert(x):
    if x=="Y":
        return 1
    if x=='N':
        return 0

In [16]:
df1.Open=df1.Open.apply(yesnoconvert)

In [21]:
df1['sub']=df1['sub'].apply(yesnoconvert)

only retain needed columns for ML models

In [45]:
df1.columns

Index(['Unnamed: 0', 'trd_date', 'fund_id', 'investor_id', 'intermediaryid',
       'riskscore', 'SCRate', 'TrailerRate', 'FundRiskScore', 'FundName',
       'Price', 'Vol', 'return-N6M', 'return-HYG', 'return-IVV', 'AUM', 'Open',
       'Gender', 'age', 'income', 'counts', 'sub', 'ed_Deg or abv',
       'ed_H.Sch/Dip.', 'ed_Sec. or below', 'FRC'],
      dtype='object')

In [46]:
df2=df1.iloc[:,[5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25]]

In [47]:
df2.columns

Index(['riskscore', 'SCRate', 'TrailerRate', 'Price', 'Vol', 'return-N6M',
       'return-HYG', 'return-IVV', 'AUM', 'Open', 'Gender', 'age', 'income',
       'counts', 'sub', 'ed_Deg or abv', 'ed_H.Sch/Dip.', 'FRC'],
      dtype='object')

In [48]:
df2.to_csv('MLTotal.csv')

7-day record generation

In [74]:
df1.index=pd.to_datetime(df1.trd_date)

Filtering the trading records from 2020-03-12 to 2020-03-20

In [75]:
df3=df1.loc['2020-03-12':'2020-03-20']

In [76]:
df3.index=range(len(df3))

In [80]:
df3=df3.iloc[:,[5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25]]

In [81]:
df3.to_csv('7daysML.csv')